In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
from scipy.stats import pearsonr

%reload_ext autoreload
%autoreload 2
from HelperFunctions import dummify_columns,plot_corr, minibatch, subcategoryttest, subcategoryANOVA



# K mean clustering
from sklearn.cluster import KMeans
kmeans = KMeans()

from sklearn import metrics
from sklearn.metrics import silhouette_score
# Importing PCA
from sklearn.decomposition import PCA
# Making it so that we can see all columns of the dataframe

df=pd.read_csv('lean_df_3.csv')

/home/auscheng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (34,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df.head(5)

,id,loan_amnt,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,revol_util,total_acc,initial_list_status,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,total_rec_late_fee,collection_recoveries,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,open_act_il,open_il_12m,mths_since_rcnt_il,il_util,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_bc_sats,num_tl_120dpd_2m,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_il_high_credit_limit,fico,revol_frac,install_frac,mort_frac,card_frac,active_card_frac,active_revol_frac,open_revol_frac,good_acc_frac,fully_invested
0,68407277,3600.0,36 months,13.99,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,190xx,PA,5.91,0.0,Aug-2003,1.0,30.0,NaN,0.0,29.7,13.0,w,0.00,4421.723917,3600.00,821.72,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,0.0,30.0,Individual,NaN,NaN,NaN,0.0,722.0,2.0,0.0,21.0,36.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,0.0,0.0,148.0,128.0,3.0,3.0,4.0,4.0,2.0,2.0,0.0,0.0,3.0,76.9,0.0,0.0,178050.0,13734.0,677.0,0.692308,0.230769,0.076923,0.555556,0.400000,0.444444,0.444444,0.538462,True
1,68355089,24700.0,36 months,11.99,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,small_business,577xx,SD,16.06,1.0,Dec-1999,4.0,6.0,NaN,0.0,19.2,38.0,w,0.00,25679.660000,24700.00,979.66,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,0.0,NaN,Individual,NaN,NaN,NaN,0.0,0.0,1.0,0.0,19.0,73.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,0.0,0.0,113.0,192.0,2.0,2.0,2.0,0.0,0.0,13.0,0.0,0.0,2.0,97.4,0.0,0.0,314017.0,24667.0,717.0,0.710526,0.157895,0.105263,0.629630,0.294118,0.185185,0.740741,0.578947,True
2,68341763,20000.0,60 months,10.78,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,Dec-2015,Fully Paid,home_improvement,605xx,IL,10.78,0.0,Aug-2000,0.0,NaN,NaN,0.0,56.2,18.0,w,0.00,22705.924294,20000.00,2705.92,0.0,0.0,Jun-2017,15813.30,NaN,Mar-2019,0.0,NaN,Joint App,71000.0,13.85,Not Verified,0.0,0.0,1.0,0.0,19.0,73.0,2.0,2081.0,65.0,14000.0,2.0,5.0,1.0,0.0,0.0,125.0,184.0,14.0,14.0,101.0,10.0,0.0,2.0,0.0,0.0,0.0,100.0,0.0,0.0,218418.0,14877.0,697.0,0.388889,0.333333,0.277778,0.571429,0.500000,0.428571,0.571429,0.333333,True
3,66310712,35000.0,60 months,14.85,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,debt_consolidation,076xx,NJ,17.06,0.0,Sep-2008,0.0,NaN,NaN,0.0,11.6,17.0,w,15897.65,31464.010000,19102.35,12361.66,0.0,0.0,Feb-2019,829.90,Apr-2019,Mar-2019,0.0,NaN,Individual,NaN,NaN,NaN,0.0,0.0,1.0,0.0,23.0,70.0,1.0,6987.0,45.0,67300.0,0.0,1.0,0.0,0.0,0.0,36.0,87.0,2.0,2.0,2.0,NaN,0.0,8.0,0.0,0.0,1.0,100.0,0.0,0.0,381215.0,18000.0,787.0,0.764706,0.117647,0.058824,0.769231,0.400000,0.384615,0.769231,0.764706,True
4,68476807,10400.0,60 months,22.45,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,Source Verified,Dec-2015,Fully Paid,major_purchase,174xx,PA,25.37,1.0,Jun-1998,3.0,12.0,NaN,0.0,64.5,35.0,w,0.00,11740.500000,10400.00,1340.50,0.0,0.0,Jul-2016,10128.96,NaN,Mar-2018,0.0,NaN,Individual,NaN,NaN,NaN,0.0,0.0,3.0,0.0,14.0,84.0,7.0,9702.0,78.0,34000.0,2.0,1.0,3.0,0.0,0.0,128.0,210.0,4.0,4.0,4.0,1.0,0.0,5.0,0.0,0.0,4.0,96.6,0.0,0.0,439570.0,88097.0,697.0,0.542857,0.285714,0.171429,0.473684,0.444444,0.315789,0.368421,0.342857,True


In [4]:
df_object.columns

Index(['term', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'verification_status', 'issue_d', 'loan_status',
       'purpose', 'zip_code', 'addr_state', 'earliest_cr_line',
       'initial_list_status', 'last_pymnt_d', 'next_pymnt_d',
       'last_credit_pull_d', 'application_type', 'verification_status_joint'],
      dtype='object')

In [3]:
df_number = df.select_dtypes(include = 'number')
df_object = df.select_dtypes(include = 'object')
df_category = df.select_dtypes(include = 'category')
df_boolean = df.select_dtypes(include = 'bool')
df_datetime = df.select_dtypes(include = 'datetime')
df_timedelta = df.select_dtypes(include = 'timedelta')
#######################################################
nominal_var=list(df_object.columns)
ordinal_var=list(df_number.columns)
continuous_var=list(df_number.columns)
time_var=list(df_datetime.columns)

In [ ]:
# K mean clustering performed on categorical variables
# perform for train (non-test) set
nominal_var_cluster_num=[]
nominal_var_cluster_num_test=[]
for cat_var in nominal_var_processed:
    # dummifying the nominal categorical variable
    dummified_column=pd.get_dummies(x.loc[:,cat_var],prefix=cat_var, prefix_sep='__')
    df_for_cluster=pd.concat([y,dummified_column],axis=1,sort='False')
    print(cat_var)
    # Finding the optimal number of clusters and storing into nominal_var_cluster_num
    kmax = df_for_cluster.shape[1]
    KNumberChoice=range(2, kmax)
    # dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
    # Doing mulitple trials for each category
    cluster_count=[]
    for trial_num in range(0,5):
        sil = []
        for k in range(2, kmax):
            kmeans = KMeans(n_clusters = k,random_state=trial_num,init='k-means++').fit(df_for_cluster)
            labels = kmeans.labels_
            sil.append(silhouette_score(df_for_cluster, labels, metric = 'euclidean'))
        if np.argmax(sil)<0.2: # defining threshold for 1 cluster
            cluster_count.append(1)
        else:
            cluster_count.append(KNumberChoice[np.argmax(sil)])
    nominal_var_cluster_num.append(max(set(cluster_count), key=cluster_count.count))
print(nominal_var_cluster_num)
print(nominal_var_cluster_num_test)

In [ ]:
# With the known cluster number for each categorical variable, perform the clustering
# perform for train (non-test) set
for ind,cat_var in enumerate(nominal_var_processed):
    dummified_column=pd.get_dummies(x.loc[:,cat_var],prefix=cat_var, prefix_sep='__')
    df_for_cluster=pd.concat([y,dummified_column],axis=1,sort='False')
    kmeans=KMeans(n_clusters=nominal_var_cluster_num[ind]).fit(df_for_cluster)
    x.loc[:,cat_var]=kmeans.labels_
    
clustered_df=pd.concat([y,x],axis=1,sort='False')    
clustered_df_test=pd.concat([y_test,x_test],axis=1,sort='False')    
print(clustered_df.shape)
print(clustered_df_test.shape)

In [ ]:
# Removing the columns with pure 0's
undummified_clustered_df=clustered_df.loc[:,(clustered_df != 0).any(axis=0)]
undummified_clustered_df.shape

In [ ]:
####################### ONLY FOR NON-TEST !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
####################### ONLY FOR NON-TEST !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Removing sub-categories with low counts and their associated observations (if less than 2% of total observation)
boolmatrix=[]
non_cont_columns=dummified_clustered_df.columns[~dummified_clustered_df.columns.isin(cont_var_for_tuning)]
for cat_feature in non_cont_columns:
    if dummified_clustered_df.loc[:,cat_feature].sum()<0.020*dummified_clustered_df.shape[0]:
        print('yes')
        boolvec=(dummified_clustered_df.loc[:,cat_feature]==1)
        boolmatrix.append(list(boolvec))
   

In [ ]:
####################### ONLY FOR NON-TEST !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
####################### ONLY FOR NON-TEST !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# creating boolean vector to takeout observations that have categorical low count observations
if boolmatrix!=[]:
    reduction_bool_vec=np.any(np.array(boolmatrix).transpose(),axis=1)
    temp_df=dummified_clustered_df[~reduction_bool_vec]
    purged_dummified_clustered_df=temp_df.loc[:,(temp_df != 0).any(axis=0)]
else:
    purged_dummified_clustered_df=dummified_clustered_df
print(purged_dummified_clustered_df.shape)
print(dummified_clustered_df.shape)

In [ ]:
# Removing from the test set columns there were removed from the training set!
cols_to_keep=set(dummified_clustered_df_test.columns)&(set(purged_dummified_clustered_df.columns))


purged_dummified_clustered_df_test=dummified_clustered_df_test.loc[:,list(cols_to_keep)]
purged_dummified_clustered_df=dummified_clustered_df.loc[:,list(cols_to_keep)+['SalePrice']]

# purged_dummified_clustered_df.reset_index(inplace=True)
# purged_dummified_clustered_df.set_index('Id')
# purged_dummified_clustered_df_test.reset_index(inplace=True)
# purged_dummified_clustered_df_test.set_index('Id')
# purged_dummified_clustered_df

# purged_dummified_clustered_df.sample(20)

In [ ]:
def undummify(dataframe):
    tot_col=dataframe.columns
    cat_col=list(tot_col[tot_col.str.contains('__')])
    cat_col_split=set(map(lambda x:x.split('__')[0],cat_col))
    cat_dict={}
    for col in cat_col_split:
        sub_df=dataframe[cat_col].loc[:,list(map(lambda x:col in x, dataframe[cat_col].columns))]
        for i in sub_df.columns:
            label_num=int(i.split('__')[1])
            sub_df.loc[:,i]=np.array(sub_df.loc[:,i])*label_num
        cat_dict[col]=sub_df.sum(axis=1)+1
    df1=dataframe.drop(cat_col,axis=1)
    df2=pd.DataFrame(cat_dict)
    return pd.concat([df1,df2],axis=1)

In [11]:
import numpy as np
from kmodes.kmodes import KModes

# random categorical data
data = np.random.choice(20, (100, 10))

km = KModes(n_clusters=4, init='Huang', n_init=5, verbose=1)

clusters = km.fit_predict(data)

# Print the cluster centroids
print(km.cluster_centroids_)

[autoreload of HelperFunctions failed: Traceback (most recent call last):
  File "/home/auscheng/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/auscheng/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/home/auscheng/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/home/auscheng/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "/home/auscheng/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 317, in update_instances
    update_instances(old, new, obj, visited)
  File "/home/auscheng/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 317, in update_instances
    update_instan

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 17, cost: 803.0
Run 1, iteration: 2/100, moves: 10, cost: 799.0
Run 1, iteration: 3/100, moves: 4, cost: 799.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 19, cost: 794.0
Run 2, iteration: 2/100, moves: 3, cost: 794.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 21, cost: 806.0
Run 3, iteration: 2/100, moves: 7, cost: 806.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 21, cost: 801.0
Run 4, iteration: 2/100, moves: 5, cost: 798.0
Run 4, iteration: 3/100, moves: 1, cost: 797.0
Run 4, iteration: 4/100, moves: 0, cost: 797.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 20, cost: 796.0
Run 5, iteration: 2/100, moves

In [12]:
km

array([[13,  2,  5, 19,  1, 19,  7,  6,  0,  7],
       [ 0,  1,  0, 16, 19,  4, 10,  9,  5,  4],
       [14,  5, 19,  7,  5,  1,  1,  9,  4,  2],
       [14, 13, 14, 16, 14, 15,  2,  4, 15,  0],
       [ 6,  0, 19,  0,  8, 15, 12,  3,  3,  2],
       [ 1,  7, 18,  1, 17,  7,  8,  0, 11,  8],
       [15,  6, 15, 15,  2,  4,  1,  2,  3,  5],
       [ 6,  7,  4,  2,  5,  5,  7, 14,  5,  1],
       [13, 14,  2, 10, 17, 17, 18,  7,  9,  7],
       [ 7,  1, 14,  9, 14,  4, 18,  7,  0, 13],
       [16,  8,  3,  7, 14,  1,  8, 19,  2, 17],
       [13, 14,  2,  2, 10,  8, 10, 18,  6,  5],
       [ 3,  6, 17,  8, 12, 11,  3, 12,  4,  0],
       [13,  3, 11,  0, 12,  2,  8, 12, 18, 19],
       [ 4, 12,  3,  2,  4, 18,  8,  6,  6,  2],
       [14,  5, 16, 18,  0, 11, 14,  6, 17, 19],
       [ 7, 12, 13, 16, 19, 14, 11, 13, 17,  3],
       [ 2,  3,  8, 13,  9,  2,  7,  8, 19, 17],
       [19,  3, 17,  8, 17,  5,  6,  5,  7,  4],
       [17, 18, 17,  9,  6, 19, 11, 14, 16, 12],
       [ 3,  5, 15, 